In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, BatchNormalization, Conv2D, MaxPooling2D, Flatten, 
                                     GRU, Dropout, Input, Embedding, LayerNormalization, 
                                     MultiHeadAttention, GlobalAveragePooling1D)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.datasets import mnist, imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# =============================================================================
# 1. Klasyfikacja IRIS z siecią gęstą i BatchNormalization
# =============================================================================
print("\n" + "="*80)
print("1. Klasyfikacja IRIS z BatchNormalization")
print("="*80)

# Wczytanie i przygotowanie danych
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# Kodowanie one-hot i podział danych
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Normalizacja danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Budowa modelu
model_iris = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(3, activation='softmax')
])

# Kompilacja i trening modelu
model_iris.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_iris = model_iris.fit(X_train, y_train, epochs=100, batch_size=16, 
                             validation_split=0.2, verbose=0)

# Ocena modelu
test_loss_iris, test_acc_iris = model_iris.evaluate(X_test, y_test, verbose=0)
print(f'Dokładność modelu IRIS: {test_acc_iris:.4f}')

# =============================================================================
# 2. Rozpoznawanie cyfr MNIST z konwolucjami 5x5
# =============================================================================
print("\n" + "="*80)
print("2. Klasyfikacja MNIST z jądrami 5x5")
print("="*80)

# Wczytanie i przygotowanie danych
(X_train_mnist, y_train_mnist), (X_test_mnist, y_test_mnist) = mnist.load_data()
X_train_mnist = X_train_mnist.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test_mnist = X_test_mnist.reshape(-1, 28, 28, 1).astype('float32') / 255.0

y_train_mnist = to_categorical(y_train_mnist, 10)
y_test_mnist = to_categorical(y_test_mnist, 10)

# Budowa modelu
model_mnist = Sequential([
    Conv2D(32, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, kernel_size=(5,5), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Kompilacja i trening modelu
model_mnist.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_mnist = model_mnist.fit(X_train_mnist, y_train_mnist, epochs=5, batch_size=128, 
                               validation_split=0.1, verbose=0)

# Ocena modelu
test_loss_mnist, test_acc_mnist = model_mnist.evaluate(X_test_mnist, y_test_mnist, verbose=0)
print(f'Dokładność modelu MNIST: {test_acc_mnist:.4f}')

# =============================================================================
# 3. Prognozowanie sekwencji z GRU i Dropout
# =============================================================================
print("\n" + "="*80)
print("3. Prognozowanie sekwencji z GRU i Dropout")
print("="*80)

# Generowanie sztucznych danych
n_samples = 10000
seq_length = 20
n_features = 5

X_seq = np.random.randn(n_samples, seq_length, n_features)
y_seq = np.random.randn(n_samples, 1)  # Symulacja zadania regresji

# Podział danych
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

# Budowa modelu
model_seq = Sequential([
    GRU(64, input_shape=(seq_length, n_features), return_sequences=False),
    Dropout(0.2),
    Dense(1)  # Warstwa wyjściowa dla regresji
])

# Kompilacja i trening modelu
model_seq.compile(optimizer='adam', loss='mse')
history_seq = model_seq.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, 
                           validation_split=0.2, verbose=0)

# Ocena modelu
test_mse_seq = model_seq.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f'Błąd MSE modelu sekwencyjnego: {test_mse_seq:.4f}')

# =============================================================================
# 4. Analiza sentymentu IMDB z Transformer Encoder
# =============================================================================
print("\n" + "="*80)
print("4. Analiza sentymentu IMDB z Transformer Encoder")
print("="*80)

# Parametry
max_features = 20000
max_len = 200

# Wczytanie i przygotowanie danych
(X_train_imdb, y_train_imdb), (X_test_imdb, y_test_imdb) = imdb.load_data(num_words=max_features)
X_train_imdb = pad_sequences(X_train_imdb, maxlen=max_len)
X_test_imdb = pad_sequences(X_test_imdb, maxlen=max_len)

# Funkcja budująca blok Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalizacja i mechanizm uwagi
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = tf.keras.layers.Add()([x, inputs])
    
    # Feed Forward Network
    y = LayerNormalization(epsilon=1e-6)(x)
    y = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(y)
    y = tf.keras.layers.Dropout(dropout)(y)
    y = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(y)
    return tf.keras.layers.Add()([y, x])

# Budowa modelu
inputs = Input(shape=(max_len,))
x = Embedding(max_features, 128)(inputs)
x = transformer_encoder(x, head_size=128, num_heads=2, ff_dim=256, dropout=0.1)
x = transformer_encoder(x, head_size=128, num_heads=2, ff_dim=256, dropout=0.1)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
outputs = Dense(1, activation='sigmoid')(x)

model_imdb = Model(inputs, outputs)

# Kompilacja i trening modelu
model_imdb.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_imdb = model_imdb.fit(X_train_imdb, y_train_imdb, epochs=3, batch_size=32, 
                             validation_split=0.2, verbose=0)

# Ocena modelu
test_loss_imdb, test_acc_imdb = model_imdb.evaluate(X_test_imdb, y_test_imdb, verbose=0)
print(f'Dokładność modelu IMDB: {test_acc_imdb:.4f}')

# =============================================================================
# Podsumowanie wyników
# =============================================================================
print("\n" + "="*80)
print("Podsumowanie wszystkich modeli")
print("="*80)
print(f"1. IRIS - dokładność: {test_acc_iris:.4f}")
print(f"2. MNIST - dokładność: {test_acc_mnist:.4f}")
print(f"3. Sekwencyjny - MSE: {test_mse_seq:.4f}")
print(f"4. IMDB - dokładność: {test_acc_imdb:.4f}")

ImportError: Traceback (most recent call last):
  File "D:\ProgramData\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.